# Segment 1 Lab 2

## Making our own Customer Support Chatbot

One of the most common business use cases of Gen AI.

We'll even make our own User Interface - no frontend skills required!

We will use the delightful `gradio` framework which makes it remarkably easy for data scientists to build great UIs.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = {"role": "system", "content":"You are a helpful assistant"}

Reminder of the structure of prompt messages to OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```



## The chat function

In order to use Gradio's out-of-the-box Chat User Interfacem we need to write a single function, `chat(message, history)`

In [ ]:
def chat(message, history):
    messages = [system_message] + history + [{"role":"user", "content":message}]
    results = openai.chat.completions.create(model=MODEL, messages=messages)
    return results.choices[0].message.content

## And then enter Gradio's magic!

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

# Making a shopping assistant

This is an example of "multi-shot prompting" - we give the model examples of conversations so it picks up how to respond:

In [ ]:
system_text = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'\
Encourage the customer to buy hats if they are unsure what to get."
system_message = {"role": "system", "content": system_text}

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

In [ ]:
system_message["content"] += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
system_message["content"] =  """You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so."""

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4c - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

# Bringing it together

This is the start of what you might call an "agent framework", in that we will use multiple LLM calls to solve a complex problem.

We'll work on a full agent framework in the final project today!

In [ ]:
def chat(history):
    message = history[-1]["content"]
    messages = [system_message] + history
    results = openai.chat.completions.create(model=MODEL, messages=messages)
    image = artist("London") if "London" in message else None
    response = results.choices[0].message.content
    history += [{"role":"assistant", "content":response}]
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)